# House Requests and 311 Efficiency
In this notebook, we go on with the analysis by trying to detect which neighborhoods guarantee a more comfortable life, according to different factors that we'll step after step identify and describe.

## Implementation plan
* Extract relevant request categories from the main dataset;
* Time series analysis: period of the year, time of day, call duration, with clear comparison over the years (this is a key point of this analysis, to provide a measure of how much has the 311 efficiency improved/got worse over the years);
* Geolocation and maps: distinguish between boroughs

### Library imports

In [1]:
from datetime import datetime, timedelta
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
#import mpld3
%matplotlib inline

### Load the dataset

In [3]:
# Load it into a Dataframe using pandas
path = '../data/311_filtered_2.csv'
df = pd.read_csv(path)
df.head()

,Created Date,Closed Date,Agency,Complaint Type,Descriptor,Location Type,Borough,Latitude,Longitude
0,11/07/2011 04:02:50 PM,11/07/2011 05:45:14 PM,NYPD,Blocked Driveway,No Access,Street/Sidewalk,BROOKLYN,40.609242,-73.982821
1,11/07/2011 11:53:37 AM,11/07/2011 03:14:40 PM,NYPD,Illegal Parking,Double Parked Blocking Traffic,Street/Sidewalk,BROOKLYN,40.692605,-73.997137
2,11/07/2011 05:53:47 PM,11/07/2011 06:13:21 PM,NYPD,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,MANHATTAN,40.762290,-73.986106
3,11/07/2011 07:47:15 PM,11/08/2011 12:13:05 AM,NYPD,Illegal Parking,Unauthorized Bus Layover,Street/Sidewalk,MANHATTAN,40.726866,-73.979908
4,11/07/2011 09:11:39 PM,11/08/2011 04:53:53 AM,DOT,Broken Muni Meter,No Receipt,Street,MANHATTAN,40.759064,-73.978450


We group by `Complaint Type` (which represents the category of the request) and sort by the most frequent complaint types:

In [5]:
# The 50 most frequent crimes
call_freq = df['Complaint Type'].groupby(df['Complaint Type']).count().sort_values(ascending=False).reset_index(name='count')
call_freq

,Complaint Type,count
0,Noise - Residential,921443
1,HEAT/HOT WATER,666588
2,HEATING,518605
3,Blocked Driveway,425798
4,PLUMBING,364728
5,Illegal Parking,363760
6,GENERAL CONSTRUCTION,258868
7,Dirty Conditions,235794
8,UNSANITARY CONDITION,225624
9,PAINT/PLASTER,197093


As we can see, there are many categories that have a very low count, therefore let's try to filter them out by picking those complaint types with high frequency (>= 10000):

In [6]:
call_freq[call_freq['count']>=10000]

,Complaint Type,count
0,Noise - Residential,921443
1,HEAT/HOT WATER,666588
2,HEATING,518605
3,Blocked Driveway,425798
4,PLUMBING,364728
5,Illegal Parking,363760
6,GENERAL CONSTRUCTION,258868
7,Dirty Conditions,235794
8,UNSANITARY CONDITION,225624
9,PAINT/PLASTER,197093


In [8]:
df[df['Complaint Type']=='GENERAL CONSTRUCTION'].head(100)

,Created Date,Closed Date,Agency,Complaint Type,Descriptor,Location Type,Borough,Latitude,Longitude
24,11/07/2011 12:00:00 AM,11/21/2011 12:00:00 AM,HPD,GENERAL CONSTRUCTION,CABINETS,RESIDENTIAL BUILDING,BRONX,40.822364,-73.897858
25,11/07/2011 12:00:00 AM,11/21/2011 12:00:00 AM,HPD,GENERAL CONSTRUCTION,DOORS,RESIDENTIAL BUILDING,QUEENS,40.756481,-73.855739
28,11/07/2011 12:00:00 AM,11/28/2011 12:00:00 AM,HPD,GENERAL CONSTRUCTION,MOLD,RESIDENTIAL BUILDING,BRONX,40.851999,-73.922515
29,11/07/2011 12:00:00 AM,12/05/2011 12:00:00 AM,HPD,GENERAL CONSTRUCTION,LOCKS,RESIDENTIAL BUILDING,MANHATTAN,40.825661,-73.948648
35,11/07/2011 12:00:00 AM,12/12/2011 12:00:00 AM,HPD,GENERAL CONSTRUCTION,WINDOW GUARDS,RESIDENTIAL BUILDING,BROOKLYN,40.700719,-73.917350
37,11/07/2011 12:00:00 AM,11/16/2011 12:00:00 AM,HPD,GENERAL CONSTRUCTION,DOORS,RESIDENTIAL BUILDING,QUEENS,40.689920,-73.791095
43,11/07/2011 12:00:00 AM,11/16/2011 12:00:00 AM,HPD,GENERAL CONSTRUCTION,DOORS,RESIDENTIAL BUILDING,QUEENS,40.748646,-73.911401
45,11/07/2011 12:00:00 AM,11/28/2011 12:00:00 AM,HPD,GENERAL CONSTRUCTION,WINDOWS,RESIDENTIAL BUILDING,MANHATTAN,40.853720,-73.932114
51,11/07/2011 12:00:00 AM,11/23/2011 12:00:00 AM,HPD,GENERAL CONSTRUCTION,DOORS,RESIDENTIAL BUILDING,BROOKLYN,40.676918,-73.893279
53,11/07/2011 12:00:00 AM,11/22/2011 12:00:00 AM,HPD,GENERAL CONSTRUCTION,FLOOR,RESIDENTIAL BUILDING,QUEENS,40.602244,-73.753957
